In [10]:
import sys
import os
import pandas as pd

if 'google.colab' in sys.modules:
    # Клонуємо репозиторій (lab-02)
    if not os.path.exists('/content/NLP'):
        !git clone -b lab-02 https://github.com/AndrianaNahirna/NLP.git

    %cd /content/NLP
    !pip install -r requirements.txt -q

    sys.path.append('/content/NLP')
    FOLDER_ID = '1Z4ko8PYcLJOnnU98T6MTXLVYHnpMkHVK'
    os.makedirs('/content/NLP/data', exist_ok=True)
    !gdown --folder https://drive.google.com/drive/folders/{FOLDER_ID} -O /content/NLP/data/

    # Шлях до сирого датасету
    raw_data_path = '/content/NLP/data/NLP_datasets/raw.csv'
else:
    # Для локальної роботи
    sys.path.append(os.path.abspath('.'))
    raw_data_path = 'data/raw.csv'

# Завантаження даних
try:
    df_raw = pd.read_csv(raw_data_path)
    print(f"\nДатасет успішно завантажено. Кількість рядків: {len(df_raw)}")
except Exception as e:
    print(f"Помилка: {e}. Перевірте назву файлу в папці data")

/content/NLP
Retrieving folder contents
Processing file 17odn4ukdHLvZKqqUuaTNPuHX66Aal-zk processed_v2.csv
Processing file 1tVj7OaRkYqaoVtmDGgDxUQ8nkDUvy7W7 raw.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=17odn4ukdHLvZKqqUuaTNPuHX66Aal-zk
To: /content/NLP/data/NLP_datasets/processed_v2.csv
100% 5.67M/5.67M [00:00<00:00, 110MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tVj7OaRkYqaoVtmDGgDxUQ8nkDUvy7W7
To: /content/NLP/data/NLP_datasets/raw.csv
100% 5.96M/5.96M [00:00<00:00, 59.6MB/s]
Download completed
Датасет успішно завантажено. Кількість рядків: 3034


In [15]:
import requests
import json
import numpy as np
from google.colab import files

In [12]:
# Завантаження edge cases
edge_cases_path = 'sentiment/tests/edge_cases.jsonl'
df_test = pd.read_json(edge_cases_path, lines=True)

print(f"Завантажено кейсів: {len(df_test)}")

Завантажено кейсів: 20


In [13]:
from sentiment.src.preprocess import TextPreprocessor
preprocessor = TextPreprocessor()

In [14]:
# Очищення даних
df_raw['processed_text'] = df_raw['text'].astype(str).apply(
    lambda text: preprocessor.preprocess(text)['clean_normalized']
)

df_processed = df_raw[['processed_text', 'target']]

print(f"Готово. Дані очищено.")

Готово. Дані очищено.


In [16]:
# Збереження очищених даних
output_dir = '/content/NLP/data/NLP_datasets'
output_filename = 'processed_v2.csv'
final_path = os.path.join(output_dir, output_filename)

df_processed.to_csv(final_path, index=False)

print(f"Файл збережено за шляхом: {final_path}")

Файл збережено за шляхом: /content/NLP/data/NLP_datasets/processed_v2.csv


In [17]:
# Приклади raw -> processed_v2
sample_examples = df_raw.sample(n=15, random_state=42)

print("-"*200)
print("15 прикладів 'до' та 'після' препроцесингу")
print("-"*200)

for idx, row in sample_examples.iterrows():
    print(f"\nІндекс у датасеті: {idx}")
    print(f"RAW       : {row['text']}")
    print(f"PROCESSED : {row['processed_text']}\n")
    print("-" * 200)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
15 прикладів 'до' та 'після' препроцесингу
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Індекс у датасеті: 2880
RAW       : 1. Товар компанією "РОЗЕТКА. УА" відправлений без перевірки на працездатність.
При отриманні мною виявлено, що медіаплеєр ARTLINE TvBox KM3 4 / 64GB Android TV 9.0 знаходиться не в робочому стані і експлуатуватися не може так як не видає картинку на екран телевізора у включеному режимі.
2. Тривала затримка в ухваленні рішення продавцем про заміну товару на аналогічний належної якості.
З 26.12.2019 р медіаплеєр ARTLINE TvBox KM3 4 / 64GB Android TV 9.0 знаходиться в сервісному центрі компанії "РОЗЕТКА. УА" та по 1

In [18]:
# Статистика
df_stats = df_raw.copy()

# Кількість порожніх / дуже коротких текстів (< 3 символів)
df_stats['text'] = df_stats['text'].fillna('')
df_stats['processed_text'] = df_stats['processed_text'].fillna('')

short_raw = len(df_stats[df_stats['text'].str.strip().str.len() < 3])
short_proc = len(df_stats[df_stats['processed_text'].str.strip().str.len() < 3])

# Кількість точних дублікатів
total_rows = len(df_stats)
dup_raw_count = df_stats.duplicated(subset=['text']).sum()
dup_proc_count = df_stats.duplicated(subset=['processed_text']).sum()

dup_raw_pct = (dup_raw_count / total_rows) * 100
dup_proc_pct = (dup_proc_count / total_rows) * 100

# Скільки замін зроблено
tags_to_count = ['<URL>', '<EMAIL>', '<PHONE>', '<ID>']
tag_counts = {}

for tag in tags_to_count:
    tag_counts[tag] = df_stats['processed_text'].str.count(tag).sum()

# Кількість символів в рядку
df_stats['char_len_raw'] = df_stats['text'].str.len()
df_stats['char_len_proc'] = df_stats['processed_text'].str.len()

# Кількість слів в рядку
df_stats['word_len_raw'] = df_stats['text'].str.split().str.len()
df_stats['word_len_proc'] = df_stats['processed_text'].str.split().str.len()


print("Статистика 'до/після'\n")

print(f"Загальна кількість записів: {total_rows}\n")

print("1. Порожні або короткі рядки (< 3 символів):")
print(f"   До    : {short_raw}")
print(f"   Після : {short_proc}\n")

print("2. Точні дублікати:")
print(f"   До    : {dup_raw_count} ({dup_raw_pct:.2f}%)")
print(f"   Після : {dup_proc_count} ({dup_proc_pct:.2f}%)\n")

print("3. Розподіл довжин (символи):")
print(f"   До    : Середня={df_stats['char_len_raw'].mean():.1f}, Медіана={df_stats['char_len_raw'].median():.1f}, Мін={df_stats['char_len_raw'].min()}, Макс={df_stats['char_len_raw'].max()}")
print(f"   Після : Середня={df_stats['char_len_proc'].mean():.1f}, Медіана={df_stats['char_len_proc'].median():.1f}, Мін={df_stats['char_len_proc'].min()}, Макс={df_stats['char_len_proc'].max()}\n")

print("4. Розподіл довжин (слова):")
print(f"   До    : Середня={df_stats['word_len_raw'].mean():.1f}, Медіана={df_stats['word_len_raw'].median():.1f}, Мін={df_stats['word_len_raw'].min()}, Макс={df_stats['word_len_raw'].max()}")
print(f"   Після : Середня={df_stats['word_len_proc'].mean():.1f}, Медіана={df_stats['word_len_proc'].median():.1f}, Мін={df_stats['word_len_proc'].min()}, Макс={df_stats['word_len_proc'].max()}\n")

print("5. Зроблені заміни:")
for tag, count in tag_counts.items():
    print(f"   {tag:<8} : {int(count)} разів")

Статистика 'до/після'

Загальна кількість записів: 3034

1. Порожні або короткі рядки (< 3 символів):
   До    : 0
   Після : 0

2. Точні дублікати:
   До    : 9 (0.30%)
   Після : 9 (0.30%)

3. Розподіл довжин (символи):
   До    : Середня=1096.7, Медіана=897.0, Мін=514, Макс=6070
   Після : Середня=1034.4, Медіана=835.0, Мін=378, Макс=5960

4. Розподіл довжин (слова):
   До    : Середня=164.0, Медіана=132.0, Мін=66, Макс=905
   Після : Середня=161.1, Медіана=130.0, Мін=64, Макс=903

5. Зроблені заміни:
   <URL>    : 107 разів
   <EMAIL>  : 30 разів
   <PHONE>  : 37 разів
   <ID>     : 431 разів


In [19]:
# Обробка edge cases
df_test['actual_result'] = df_test['raw_text'].apply(lambda x: preprocessor.preprocess(x))

print(f"Успішно завантажено та оброблено {len(df_test)} кейсів\n")
print("-" * 200)

for _, row in df_test.sample(5, random_state=1).iterrows():
    print(f"ID: {row['id']}")
    print(f"INPUT:  {row['raw_text']}")
    print(f"RESULT: {row['actual_result'].get('clean_normalized', 'Ключ не знайдено')}")
    print(f"EXPECT: {row['expected_behavior']}")
    print("-" * 200)

Успішно завантажено та оброблено 20 кейсів

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
ID: 4
INPUT:  Волею долі купував в Розетці тільки освітлювання прилади (замовлення №№ 425796, 425797, 1348674), хоча задекларований асортимент товарів вражає уяву. Оплата - ніяких передоплат (принаймні для доставки до Дніпропетровська). Доставка в регіони не сама оперативна (в Дніпропетровськ 4-5 днів). Розгорнутим згорнути
RESULT: Волею долі купував в Розетці тільки освітлювання прилади (замовлення <ID> ), хоча задекларований асортимент товарів вражає уяву. Оплата - ніяких передоплат (принаймні для доставки до Дніпропетровська). Доставка в регіони не сама оперативна (в Дніпропетровськ 4-5 днів).
EXPECT: Маскування номерів замовлень як ідентифікаторів, очищення зайвих символів переносу рядка, видалення технічного тексту 'Розгорнутим згорнути'.
-

In [21]:
# Перевірка Idempotence
# Беремо 100 випадкових текстів з датасету
sample_texts = df_raw['text'].dropna().sample(100, random_state=42)
idempotence_passed = True
failed_cases = []

for text in sample_texts:
    # Проганяємо raw
    step1 = preprocessor.preprocess(text)['clean_normalized']

    # Проганяємо processed
    step2 = preprocessor.preprocess(step1)['clean_normalized']

    # Вони мають бути ідентичними
    if step1 != step2:
        idempotence_passed = False
        failed_cases.append((step1, step2))

if idempotence_passed:
    print("Idempotence: пройдено")
    print("preprocess(preprocess(x)) == preprocess(x)")
else:
    print(f"Idempotence: провалено на {len(failed_cases)} кейсах.")
    print(f"Приклад розбіжності:\n   Прогін 1: {failed_cases[0][0]}\n   Прогін 2: {failed_cases[0][1]}")


# Перевірка No empty explosions
# Шукаємо рядки, які були достатньо довгими (> 10 символів),
# але після нашого пайплайну стали повністю порожніми (0 символів)
explosions = df_raw[(df_raw['text'].str.len() > 10) & (df_raw['processed_text'].str.len() == 0)]

if len(explosions) == 0:
    print("\nNo empty explosions: пройдено")
    print("Жоден змістовний текст не перетворився на порожній рядок.")
else:
    print(f"\nNo empty explosions: провалено.")
    print(f"Знайдено {len(explosions)} текстів, які зникли.")
    print(f"Приклад: {explosions.iloc[0]['text']}")

Idempotence: пройдено
preprocess(preprocess(x)) == preprocess(x)

No empty explosions: пройдено
Жоден змістовний текст не перетворився на порожній рядок.


In [22]:
# Генерація файлу audit_summary_lab2.md

audit_md = f"""# Audit Summary: Lab 2 (Text Cleaning & Normalization)

## 1. Загальна статистика
* **Загальна кількість записів:** {total_rows}
* **Порожні або короткі рядки (< 3 символів):** До = {short_raw}, Після = {short_proc}
* **Точні дублікати:** До = {dup_raw_count} ({dup_raw_pct:.2f}%), Після = {dup_proc_count} ({dup_proc_pct:.2f}%)

## 2. Розподіл довжин
**Символи:**
* **До:** Середня={df_stats['char_len_raw'].mean():.1f}, Медіана={df_stats['char_len_raw'].median():.1f}, Мін={df_stats['char_len_raw'].min()}, Макс={df_stats['char_len_raw'].max()}
* **Після:** Середня={df_stats['char_len_proc'].mean():.1f}, Медіана={df_stats['char_len_proc'].median():.1f}, Мін={df_stats['char_len_proc'].min()}, Макс={df_stats['char_len_proc'].max()}

**Слова:**
* **До:** Середня={df_stats['word_len_raw'].mean():.1f}, Медіана={df_stats['word_len_raw'].median():.1f}, Мін={df_stats['word_len_raw'].min()}, Макс={df_stats['word_len_raw'].max()}
* **Після:** Середня={df_stats['word_len_proc'].mean():.1f}, Медіана={df_stats['word_len_proc'].median():.1f}, Мін={df_stats['word_len_proc'].min()}, Макс={df_stats['word_len_proc'].max()}

## 3. Зроблені маскування (PII & IDs)
* `<URL>`: {int(tag_counts.get('<URL>', 0))} разів
* `<EMAIL>`: {int(tag_counts.get('<EMAIL>', 0))} разів
* `<PHONE>`: {int(tag_counts.get('<PHONE>', 0))} разів
* `<ID>`: {int(tag_counts.get('<ID>', 0))} разів

## 4. Результати тестування (Міні-регресія)
* **Idempotence:** {"Пройдено" if idempotence_passed else "Провалено"}
* **No Empty Explosions:** {"Пройдено" if len(explosions) == 0 else "Провалено"}
"""

audit_filename = "audit_summary_lab2.md"
with open(audit_filename, "w", encoding="utf-8") as f:
    f.write(audit_md)

print(f"Файл {audit_filename} згенеровано.")

if 'google.colab' in sys.modules:
    from google.colab import files
    print("Завантаження audit_summary_lab2.md на комп'ютер")
    files.download(audit_filename)

Файл audit_summary_lab2.md згенеровано.
Завантаження audit_summary_lab2.md на комп'ютер


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>